In [2]:
import os
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext, SparkConf

In [3]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import countDistinct, avg
from pyspark.sql.functions import dayofmonth,dayofyear,year,month,hour,weekofyear,date_format
from pyspark.sql.functions import col as func_col

In [4]:
cwd = os.getcwd()
for part in cwd.split('/'):
    if part.lower().startswith('edureka'):
        user_id = part.title()
user_id

'Edureka_121039'

In [5]:
app_name = '{0} : Spark SQL'.format(user_id)
app_name

'Edureka_121039 : Spark SQL'

In [6]:
# spark.sparkContext.stop()
# spark.stop()
spark = SparkSession.builder.appName(app_name).getOrCreate() # singleton instance

In [7]:
def get_hdfs_filepath(file_name):
    my_hdfs = '/user/{0}'.format(user_id.lower())
    return os.path.join(my_hdfs, file_name)

### Datasets
Ensure you save all data files under dir <font color='red'>**/user/edureka_(your-user-id)/**</font> on HDFS using Hue browser.

In [8]:
EMP_JSON = get_hdfs_filepath('emp.json')
EMP_CSV = get_hdfs_filepath('emp.csv') # comma delimited
DEPT_CSV = get_hdfs_filepath('dept.csv')

PEOPLE_JSON = get_hdfs_filepath('people.json')
GOOG_CSV = get_hdfs_filepath('goog_stock.csv')
NESTED_JSON = get_hdfs_filepath('nested.json')

In [50]:
emp_json_df = spark.read.json(EMP_JSON)

In [51]:
emp_json_df.show()

+---+---+------+------+
|age| id|  name|salary|
+---+---+------+------+
| 30|  1|  Bill| 10000|
| 40|  2| Steve| 12000|
| 50|  3|Donald| 14000|
| 60|  4|  Modi| 18000|
| 70|  5|Sunder| 22000|
| 80|  6|  Jeff| 26000|
| 90|  7|Sergey| 30000|
+---+---+------+------+



In [52]:
emp_json_df.printSchema()

root
 |-- age: long (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)



In [53]:
emp_json_df.columns

['age', 'id', 'name', 'salary']

In [54]:
emp_json_df.describe()

DataFrame[summary: string, age: string, id: string, name: string, salary: string]

In [55]:
emp_json_df.describe().show()

+-------+------------------+-----------------+------+-----------------+
|summary|               age|               id|  name|           salary|
+-------+------------------+-----------------+------+-----------------+
|  count|                 7|                7|     7|                7|
|   mean|              60.0|              4.0|  null|18857.14285714286|
| stddev|21.602468994692867|2.160246899469287|  null|7470.577207252718|
|    min|                30|                1|  Bill|            10000|
|    max|                90|                7|Sunder|            30000|
+-------+------------------+-----------------+------+-----------------+



In [56]:
emp_json_df['salary']

Column<salary>

In [57]:
type(emp_json_df['salary'])

pyspark.sql.column.Column

### Reading nested JSON

In [18]:
nested = spark.read.json(NESTED_JSON)

In [19]:
nested.printSchema()

root
 |-- age: long (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- phones: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- reporting: struct (nullable = true)
 |    |-- manager: string (nullable = true)
 |    |-- rm: string (nullable = true)
 |-- salary: long (nullable = true)



In [20]:
phones = nested.select([explode('phones'), 'id'])
phones.show()

+-----+---+
|  col| id|
+-----+---+
|12345|  1|
|56789|  1|
+-----+---+



In [21]:
report = nested.select([nested['reporting'].alias('tmp'),nested['id'],nested['age']]).select(['tmp.*', 'id', 'age'])

In [22]:
report.show()

+-------+-----+---+---+
|manager|   rm| id|age|
+-------+-----+---+---+
|  Steve|Brian|  1| 30|
+-------+-----+---+---+



### Single Column DF

In [23]:
emp_json_df.select('salary')

DataFrame[salary: bigint]

In [24]:
emp_json_df.select('salary').show()

+------+
|salary|
+------+
| 10000|
| 12000|
| 14000|
| 18000|
| 22000|
| 26000|
| 30000|
+------+



In [25]:
type(emp_json_df.select('salary'))

pyspark.sql.dataframe.DataFrame

In [26]:
emp_json_df.head(3)

[Row(age=30, id=1, name=u'Bill', salary=10000),
 Row(age=40, id=2, name=u'Steve', salary=12000),
 Row(age=50, id=3, name=u'Donald', salary=14000)]

In [27]:
emp_json_df.show(2,truncate= True)

+---+---+-----+------+
|age| id| name|salary|
+---+---+-----+------+
| 30|  1| Bill| 10000|
| 40|  2|Steve| 12000|
+---+---+-----+------+
only showing top 2 rows



In [29]:
type(emp_json_df.head(2)[0])
emp_json_df.head(2)[0]

Row(age=30, id=1, name=u'Bill', salary=10000)

### Multiple Columns

In [30]:
emp_json_df.select(['age','name'])

DataFrame[age: bigint, name: string]

In [31]:
emp_json_df.select(['age','name']).show()

+---+------+
|age|  name|
+---+------+
| 30|  Bill|
| 40| Steve|
| 50|Donald|
| 60|  Modi|
| 70|Sunder|
| 80|  Jeff|
| 90|Sergey|
+---+------+



In [32]:
# alias column names
emp_json_df.select(emp_json_df.age.alias('emp_age'), emp_json_df.name, (emp_json_df.salary * 1.05).alias('rev_sal')).show()

+-------+------+-------+
|emp_age|  name|rev_sal|
+-------+------+-------+
|     30|  Bill|10500.0|
|     40| Steve|12600.0|
|     50|Donald|14700.0|
|     60|  Modi|18900.0|
|     70|Sunder|23100.0|
|     80|  Jeff|27300.0|
|     90|Sergey|31500.0|
+-------+------+-------+



### Adding new column

emp_json_df.['newsal'] = emp_json_df['salary'] * 1.05 # TypeError: 'DataFrame' object does not support item assignment

In [33]:
# Instead, use withColumn method to add a new column (with/without a transformation)
emp_json_df.withColumn('newsal', emp_json_df['salary'] * 1.05).show()
# OR
# emp_json_df.withColumn('newsal', emp_json_df['salary']).show()

+---+---+------+------+-------+
|age| id|  name|salary| newsal|
+---+---+------+------+-------+
| 30|  1|  Bill| 10000|10500.0|
| 40|  2| Steve| 12000|12600.0|
| 50|  3|Donald| 14000|14700.0|
| 60|  4|  Modi| 18000|18900.0|
| 70|  5|Sunder| 22000|23100.0|
| 80|  6|  Jeff| 26000|27300.0|
| 90|  7|Sergey| 30000|31500.0|
+---+---+------+------+-------+



**Note**, you may execute withColumn multiple times as it just overwrites the same column name everytime without any error.
Also, the changes made here are not permanent to DF.

In [36]:
print(emp_json_df.columns)

['age', 'id', 'name', 'salary', 'newsal']


### Save Changes to DF using 

In [58]:
# make changes permanent
emp_json_df = emp_json_df.withColumn('newsal', emp_json_df['salary'] * 1.05)
emp_json_df.show()

+---+---+------+------+-------+
|age| id|  name|salary| newsal|
+---+---+------+------+-------+
| 30|  1|  Bill| 10000|10500.0|
| 40|  2| Steve| 12000|12600.0|
| 50|  3|Donald| 14000|14700.0|
| 60|  4|  Modi| 18000|18900.0|
| 70|  5|Sunder| 22000|23100.0|
| 80|  6|  Jeff| 26000|27300.0|
| 90|  7|Sergey| 30000|31500.0|
+---+---+------+------+-------+



### Renaming Columns

In [59]:
# Simple Rename
emp_json_df.withColumnRenamed('newsal', 'revised_sal').show() # does not result in error, if col is missing in DF

+---+---+------+------+-----------+
|age| id|  name|salary|revised_sal|
+---+---+------+------+-----------+
| 30|  1|  Bill| 10000|    10500.0|
| 40|  2| Steve| 12000|    12600.0|
| 50|  3|Donald| 14000|    14700.0|
| 60|  4|  Modi| 18000|    18900.0|
| 70|  5|Sunder| 22000|    23100.0|
| 80|  6|  Jeff| 26000|    27300.0|
| 90|  7|Sergey| 30000|    31500.0|
+---+---+------+------+-----------+



### Using SQL
To use SQL queries over dataframe, you will need to register it as a temporary view:

In [66]:
emp_json_df.createOrReplaceTempView('Emp')
sql_df = spark.sql('Select * From Emp')
sql_df.show()

+---+---+------+------+-------+
|age| id|  name|salary| newsal|
+---+---+------+------+-------+
| 30|  1|  Bill| 10000|10500.0|
| 40|  2| Steve| 12000|12600.0|
| 50|  3|Donald| 14000|14700.0|
| 60|  4|  Modi| 18000|18900.0|
| 70|  5|Sunder| 22000|23100.0|
| 80|  6|  Jeff| 26000|27300.0|
| 90|  7|Sergey| 30000|31500.0|
+---+---+------+------+-------+



In [41]:
# more complex ops
result = spark.sql('Select salary, salary*1.1 as rev_sal From emp Where salary >= 18000')
result.show()

+------+-------+
|salary|rev_sal|
+------+-------+
| 18000|19800.0|
| 22000|24200.0|
| 26000|28600.0|
| 30000|33000.0|
+------+-------+



<font color=red size=5>******</font> Leverage your SQL knowledge over Spark SQL and do complex operations more quickly.

### Dataframe Operations

In [35]:
# Let Spark know about the header and infer the Schema types!
# This is only available as option on CSVs and not on JSON files.
goog_df = spark.read.csv(GOOG_CSV,inferSchema=True,header=True)

In [43]:
goog_df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [44]:
goog_df.head(3)

[Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039),
 Row(Date=datetime.datetime(2010, 1, 5, 0, 0), Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002),
 Row(Date=datetime.datetime(2010, 1, 6, 0, 0), Open=214.379993, High=215.23, Low=210.750004, Close=210.969995, Volume=138040000, Adj Close=27.333178000000004)]

#### Filtering Data

In [45]:
# Using SQL
goog_df.filter("Close>200").show()

+--------------------+------------------+------------------+------------------+------------------+---------+------------------+
|                Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+--------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:...|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:...|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:...|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:...|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:...|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902

In [46]:
# Total records
goog_df.count()

1762

In [47]:
# Using SQL with select
goog_df.filter("Close>200").select(['Open','Volume']).show()

+------------------+---------+
|              Open|   Volume|
+------------------+---------+
|        213.429998|123432400|
|        214.599998|150476200|
|        214.379993|138040000|
|            211.75|119282800|
|        210.299994|111902700|
|212.79999700000002|115557400|
|209.18999499999998|148614900|
|        207.870005|151473000|
|210.11000299999998|108223500|
|210.92999500000002|148516900|
|        208.330002|182501900|
|        214.910006|153038200|
|        212.079994|152038600|
|202.51000200000001|266424900|
|205.95000100000001|466777500|
|        206.849995|430642100|
|        198.109995|163867200|
|        201.940002|135934400|
|        204.190001|109099200|
|        201.629995|105706300|
+------------------+---------+
only showing top 20 rows



#### Complex filtering using Python operators for comparison
Syntax looks very similar to SQL operators, except we need to ensure that we call the entire column within the dataframe, using the format: df["column name"]

In [140]:
# Common mistake with syntax : will produce an error, read the error to address issue here!
goog_df.filter(goog_df["Close"] < 200 and goog_df['Open'] > 200).show()

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [48]:
# Resolution: add in the parenthesis separating the statements.
goog_df.filter( (goog_df["Close"] < 200) & (goog_df['Open'] > 200)).show()

+--------------------+------------------+----------+----------+----------+---------+------------------+
|                Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+--------------------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22 00:00:...|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28 00:00:...|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29 00:00:...|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+--------------------+------------------+----------+----------+----------+---------+------------------+



#### Conditional Operators
* |   -> or
* &   -> and
* ~   -> not (equivalent to ! in Python)

In [142]:
# goog_df.filter( (goog_df["Close"] < 200) | (goog_df['Open'] > 200) ).show()
# goog_df.filter( (goog_df["Close"] < 200) & ~(goog_df['Open'] < 200) ).show()

In [49]:
# Specific value search
goog_df.filter(goog_df["Low"] == 197.16).show()

+--------------------+------------------+----------+------+------+---------+---------+
|                Date|              Open|      High|   Low| Close|   Volume|Adj Close|
+--------------------+------------------+----------+------+------+---------+---------+
|2010-01-22 00:00:...|206.78000600000001|207.499996|197.16|197.75|220441900|25.620401|
+--------------------+------------------+----------+------+------+---------+---------+



In [51]:
result = goog_df.filter(goog_df["Low"] == 197.16).collect()

In [52]:
type(result[0])
type(result)

list

Rows can be converted to a dictionary

In [53]:
row = result[0]

In [54]:
row.asDict()

{'Adj Close': 25.620401,
 'Close': 197.75,
 'Date': datetime.datetime(2010, 1, 22, 0, 0),
 'High': 207.499996,
 'Low': 197.16,
 'Open': 206.78000600000001,
 'Volume': 220441900}

In [55]:
# get all values
for item in row:
    print(item)

2010-01-22 00:00:00
206.780006
207.499996
197.16
197.75
220441900
25.620401


### GroupBy, Agg

Read employee data from CSV file

In [9]:
cemp_df = spark.read.csv(EMP_CSV,inferSchema=True,header=True)

In [10]:
cemp_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- dept_id: string (nullable = true)



In [11]:
cemp_df.show()

+---+------+---+------+-------+
| id|  name|age|salary|dept_id|
+---+------+---+------+-------+
|  1|  Bill| 30| 10000|     D1|
|  2| Steve| 40| 12000|     D2|
|  3|Donald| 50| 14000|     D2|
|  4|  Modi| 60| 18000|     D1|
|  5|Sundar| 70| 22000|     D3|
|  6|  Jeff| 80| 22000|     D3|
|  7|Sergey| 90| 30000|     D3|
+---+------+---+------+-------+



Let's group together by dept!

In [12]:
cemp_df.groupBy("dept_id")

This returns a GroupedData object, of which various methods can be invoked.

In [13]:
# Mean
cemp_df.groupBy("dept_id").mean().show()

+-------+-------+--------+------------------+
|dept_id|avg(id)|avg(age)|       avg(salary)|
+-------+-------+--------+------------------+
|     D1|    2.5|    45.0|           14000.0|
|     D3|    6.0|    80.0|24666.666666666668|
|     D2|    2.5|    45.0|           13000.0|
+-------+-------+--------+------------------+



In [14]:
cemp_df.groupBy("dept_id").mean('salary').show()

+-------+------------------+
|dept_id|       avg(salary)|
+-------+------------------+
|     D1|           14000.0|
|     D3|24666.666666666668|
|     D2|           13000.0|
+-------+------------------+



In [15]:
# applying sum over multiple columns at once
cemp_df.groupBy("dept_id").sum("salary", "age").show()

+-------+-----------+--------+
|dept_id|sum(salary)|sum(age)|
+-------+-----------+--------+
|     D1|      28000|      90|
|     D3|      74000|     240|
|     D2|      26000|      90|
+-------+-----------+--------+



### Alias name for aggregated measure columns

In below example, 'alias' method has no impact.

In [16]:
cemp_df.groupBy("dept_id").mean('salary').alias('avg sal').show()

+-------+------------------+
|dept_id|       avg(salary)|
+-------+------------------+
|     D1|           14000.0|
|     D3|24666.666666666668|
|     D2|           13000.0|
+-------+------------------+



For creating alias, we use **pyspark.sql.functions.col** imported as func_col above

In [17]:
cemp_df.groupBy("dept_id").mean('salary').select('dept_id',func_col("avg(salary)").alias("avgSal")).show()

+-------+------------------+
|dept_id|            avgSal|
+-------+------------------+
|     D1|           14000.0|
|     D3|24666.666666666668|
|     D2|           13000.0|
+-------+------------------+



Alternatively, **withColumnRenamed** can be used to rename an aggregate measure and generate alias.

In [18]:
cemp_df.groupBy("dept_id").mean('salary').withColumnRenamed('avg(salary)', 'avgSal').show()

+-------+------------------+
|dept_id|            avgSal|
+-------+------------------+
|     D1|           14000.0|
|     D3|24666.666666666668|
|     D2|           13000.0|
+-------+------------------+



<font color='red'>Multiple aggregations cannot be combined with group-by as independent functions.</font>

**Hint** : Try using <font color='red'>**agg**</font> instead to apply different aggregation measures on a single group.

In [19]:
# applying sum and mean together on columns at once - results in an ERROR
cemp_df.groupBy("dept_id").sum("salary", "age").mean("salary").show()

AttributeError: 'DataFrame' object has no attribute 'mean'

In [20]:
# Count
cemp_df.groupBy("dept_id").count().show()

+-------+-----+
|dept_id|count|
+-------+-----+
|     D1|    2|
|     D3|    3|
|     D2|    2|
+-------+-----+



In [21]:
# Max
cemp_df.groupBy("dept_id").max().show()

+-------+-------+--------+-----------+
|dept_id|max(id)|max(age)|max(salary)|
+-------+-------+--------+-----------+
|     D1|      4|      60|      18000|
|     D3|      7|      90|      30000|
|     D2|      3|      50|      14000|
+-------+-------+--------+-----------+



In [22]:
# Min
cemp_df.groupBy("dept_id").min().show()

+-------+-------+--------+-----------+
|dept_id|min(id)|min(age)|min(salary)|
+-------+-------+--------+-----------+
|     D1|      1|      30|      10000|
|     D3|      5|      70|      22000|
|     D2|      2|      40|      12000|
+-------+-------+--------+-----------+



In [23]:
# Sum
cemp_df.groupBy("dept_id").sum().show()

+-------+-------+--------+-----------+
|dept_id|sum(id)|sum(age)|sum(salary)|
+-------+-------+--------+-----------+
|     D1|      5|      90|      28000|
|     D3|     18|     240|      74000|
|     D2|      5|      90|      26000|
+-------+-------+--------+-----------+



Check out this link for more info on other methods:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module

### Aggregating without any Group By

Not all methods or analysis would always require a group-by.

In [24]:
# Max salary across everything
cemp_df.agg({'salary':'max'}).show()

+-----------+
|max(salary)|
+-----------+
|      30000|
+-----------+



### Another way of using Group By

Could try this using group by as well:

In [26]:
grouped = cemp_df.groupBy("dept_id")

In [27]:
# Max salary by dept
grouped.agg({"salary":'max'}).show()

+-------+-----------+
|dept_id|max(salary)|
+-------+-----------+
|     D1|      18000|
|     D3|      30000|
|     D2|      14000|
+-------+-----------+



### Group By multiple columns

In [29]:
# Group By col1, col2, col3
grouped = cemp_df.groupBy(["dept_id",'salary'])

### Apply aggregation functions on grouped data <font color='red'>using</font> "agg"

Single aggregation using **agg**

In [30]:
grouped.agg({"age":'max'}).show()

+-------+------+--------+
|dept_id|salary|max(age)|
+-------+------+--------+
|     D1| 18000|      60|
|     D2| 14000|      50|
|     D3| 30000|      90|
|     D3| 22000|      80|
|     D2| 12000|      40|
|     D1| 10000|      30|
+-------+------+--------+



Multiple aggregations using **agg**

In [28]:
grouped.agg({'age':'max', 'id': 'sum'}).show()

+-------+-------+--------+
|dept_id|sum(id)|max(age)|
+-------+-------+--------+
|     D1|      5|      60|
|     D3|     18|      90|
|     D2|      5|      50|
+-------+-------+--------+



Multiple aggregations using **agg** along with **alias** on aggregate measures

In [31]:
from pyspark.sql.functions import max as smax, min as smin, count, countDistinct

grouped.agg(smax("age").alias("maxAge"), 
            smin("age").alias("minAge"), 
            countDistinct('age').alias('distinctAge')).show()

+-------+------+------+------+-----------+
|dept_id|salary|maxAge|minAge|distinctAge|
+-------+------+------+------+-----------+
|     D1| 18000|    60|    60|          1|
|     D2| 14000|    50|    50|          1|
|     D3| 30000|    90|    90|          1|
|     D3| 22000|    80|    70|          2|
|     D2| 12000|    40|    40|          1|
|     D1| 10000|    30|    30|          1|
+-------+------+------+------+-----------+



### Apply aggregation functions directly on grouped-data <font color='red'>without</font> using "agg"

In [32]:
grouped.max('age', 'id').show()

+-------+------+--------+-------+
|dept_id|salary|max(age)|max(id)|
+-------+------+--------+-------+
|     D1| 18000|      60|      4|
|     D2| 14000|      50|      3|
|     D3| 30000|      90|      7|
|     D3| 22000|      80|      6|
|     D2| 12000|      40|      2|
|     D1| 10000|      30|      1|
+-------+------+--------+-------+



Renaming aggregate measure columns using **withColumnRenamed**

In [33]:
grouped.max('age', 'id').withColumnRenamed('max(age)', 'max_age').withColumnRenamed('max(id)', 'max_id').show()

+-------+------+-------+------+
|dept_id|salary|max_age|max_id|
+-------+------+-------+------+
|     D1| 18000|     60|     4|
|     D2| 14000|     50|     3|
|     D3| 30000|     90|     7|
|     D3| 22000|     80|     6|
|     D2| 12000|     40|     2|
|     D1| 10000|     30|     1|
+-------+------+-------+------+



<font color='red'>**Note**</font> : aggregation functions do not work on strings

In [34]:
grouped.max('age', 'name').show()

AnalysisException: u'"name" is not a numeric column. Aggregation function can only be applied on a numeric column.;'

### Spark SQL Functions
Variety of functions that we can import from pyspark.sql.functions. For details refer:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [36]:
goog_df.select(dayofmonth(goog_df['Date'])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows



In [37]:
goog_df.select(dayofyear(goog_df['Date'])).show()

+---------------+
|dayofyear(Date)|
+---------------+
|              4|
|              5|
|              6|
|              7|
|              8|
|             11|
|             12|
|             13|
|             14|
|             15|
|             19|
|             20|
|             21|
|             22|
|             25|
|             26|
|             27|
|             28|
|             29|
|             32|
+---------------+
only showing top 20 rows



In [38]:
goog_df.select(year(goog_df['Date'])).show()

+----------+
|year(Date)|
+----------+
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
+----------+
only showing top 20 rows



In [39]:
goog_df.select(month('Date')).show()

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          2|
+-----------+
only showing top 20 rows



In [40]:
goog_df.select(hour('Date')).show()

+----------+
|hour(Date)|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 20 rows



In [41]:
cemp_df.select(countDistinct("salary")).show()

+----------------------+
|count(DISTINCT salary)|
+----------------------+
|                     6|
+----------------------+



In [42]:
cemp_df.select(countDistinct("salary").alias("Distinct Salaries")).show()

+-----------------+
|Distinct Salaries|
+-----------------+
|                6|
+-----------------+



In [43]:
cemp_df.select(avg('salary')).show()

+------------------+
|       avg(salary)|
+------------------+
|18285.714285714286|
+------------------+



In [45]:
from pyspark.sql.functions import stddev
cemp_df.select(stddev("salary")).show()

+-------------------+
|stddev_samp(salary)|
+-------------------+
|  6969.320524371697|
+-------------------+



### Order By

In [46]:
# OrderBy
# Ascending
cemp_df.orderBy("salary").show()

+---+------+---+------+-------+
| id|  name|age|salary|dept_id|
+---+------+---+------+-------+
|  1|  Bill| 30| 10000|     D1|
|  2| Steve| 40| 12000|     D2|
|  3|Donald| 50| 14000|     D2|
|  4|  Modi| 60| 18000|     D1|
|  5|Sundar| 70| 22000|     D3|
|  6|  Jeff| 80| 22000|     D3|
|  7|Sergey| 90| 30000|     D3|
+---+------+---+------+-------+



In [47]:
# Descending sort
cemp_df.orderBy(cemp_df["salary"].desc()).show()

+---+------+---+------+-------+
| id|  name|age|salary|dept_id|
+---+------+---+------+-------+
|  7|Sergey| 90| 30000|     D3|
|  5|Sundar| 70| 22000|     D3|
|  6|  Jeff| 80| 22000|     D3|
|  4|  Modi| 60| 18000|     D1|
|  3|Donald| 50| 14000|     D2|
|  2| Steve| 40| 12000|     D2|
|  1|  Bill| 30| 10000|     D1|
+---+------+---+------+-------+



In [48]:
# Descending and ascending sort on multiple columns
cemp_df.orderBy(cemp_df.salary.desc(), cemp_df.name.asc()).show()

+---+------+---+------+-------+
| id|  name|age|salary|dept_id|
+---+------+---+------+-------+
|  7|Sergey| 90| 30000|     D3|
|  6|  Jeff| 80| 22000|     D3|
|  5|Sundar| 70| 22000|     D3|
|  4|  Modi| 60| 18000|     D1|
|  3|Donald| 50| 14000|     D2|
|  2| Steve| 40| 12000|     D2|
|  1|  Bill| 30| 10000|     D1|
+---+------+---+------+-------+



### Saving DF as Parquet

In [65]:
emp_json_df.write.mode('overwrite').parquet(get_hdfs_filepath('emp.parquet'))
# emp_json_df.write.parquet(get_hdfs_filepath('emp.parquet')) # expect exception if the parquet file already exists

In [ ]:
emp_json_df.show()

### Reading data from Parquet

In [63]:
pemp_df = spark.read.parquet('emp.parquet')
pemp_df.head(3)

[Row(age=30, id=1, name=u'Bill', salary=10000, newsal=10500.0),
 Row(age=40, id=2, name=u'Steve', salary=12000, newsal=12600.0),
 Row(age=50, id=3, name=u'Donald', salary=14000, newsal=14700.0)]

#### User Defined Functions

In [67]:
# setup SQLContext to register UDF
from pyspark.sql import SQLContext
sc = spark.sparkContext
sql_ctx = SQLContext(sc)

In [68]:
# register UDF
sql_ctx.registerFunction('ucase', lambda val: val.upper())

In [69]:
# regular query on temp view we created earlier.
spark.sql('Select * From emp Where salary >= 18000').show()

+---+---+------+------+-------+
|age| id|  name|salary| newsal|
+---+---+------+------+-------+
| 60|  4|  Modi| 18000|18900.0|
| 70|  5|Sunder| 22000|23100.0|
| 80|  6|  Jeff| 26000|27300.0|
| 90|  7|Sergey| 30000|31500.0|
+---+---+------+------+-------+



In [70]:
# query using registered UDF to transform output
spark.sql('Select ucase(name) uname, salary, age From emp Where salary >= 18000').show()

+------+------+---+
| uname|salary|age|
+------+------+---+
|  MODI| 18000| 60|
|SUNDER| 22000| 70|
|  JEFF| 26000| 80|
|SERGEY| 30000| 90|
+------+------+---+



In [71]:
# stops spark session
spark.stop()